In [205]:
import time

In [206]:
class board:
    def __init__(self, txtfile):
        '''
        Initializes useful fields to describe the game board layout.
        INPUTS: txt file representing the game board layout
        OUTPUTS: none
        RETURNS: none
        SIDE EFFECTS: intializes useful fields to describe the game board layout
        '''
        self.sqDim = 0
        self.ptBoard = []
        with open(txtfile, 'r') as f:
            while True:
                file_line = f.readline()
                if not file_line: 
                    break
                else:
                    self.sqDim += 1
                    row = file_line.replace("\r\n", '').split("\t")
                    for element in row:
                        self.ptBoard.append(int(element)) 
        self.size = self.sqDim**2
        self.maxPlayerTaken = []
        self.minPlayerTaken = []
        self.openPositions = [i for i in range(self.size)]
        
    def printBoard(self):
        '''
        Prints the current state of the board including open spaces and spaces conquered by each player
        INPUTS: none
        OUTPUTS: outputs the layout of the board and key positions that the player has taken.
        RETURNS: none
        SIDE EFFECTS: none
        '''
        for i in range(self.size):
            if i % 6 == 0:
                print ' '
            if i in self.maxPlayerTaken:
                print '[A]',
            elif i in self.minPlayerTaken:
                print '[B]',
            else:
                print '[ ]',
        print ' '

In [207]:
class warGame:
    def __init__(self, board):
        '''
        Initializes important statistics for the game
        INPUTS: board object for the game
        OUTPUTS: none
        RETURNS: none
        SIDE EFFECTS: Initializes statistics for the game
        '''
        self.board = board
        self.maxScore = 0
        self.minScore = 0
        self.totNodesExpandedMax = 0
        self.totNodesExpandedMin = 0
        self.totTimeMax = 0
        self.totTimeMin = 0
        self.totMovesMax = 0
        self.totMovesMin = 0
        
    def utility(self):
        '''
        Heuristic evaluation function that returns the current score based on positions that the current players have
        captured. The function is calculated by taking the max's score - min's score.
        INPUTS: none
        OUTPUTS: none
        RETURNS: current score of the player
        SIDE EFFECTS: none
        '''
        maxScore = 0
        minScore = 0
        for pos in self.board.maxPlayerTaken:
            maxScore += self.board.ptBoard[pos]
        for pos in self.board.minPlayerTaken:
            minScore += self.board.ptBoard[pos]
        return maxScore - minScore
            
    def calculateScore(self):
        '''
        Calculates score for each player based on positions the player's pieces occupy.
        INPUTS: None
        OUTPUTS: None
        RETURNS: None
        SIDE EFFECTS: Changes the score field in the warGame object of each player
        '''
        self.maxScore = 0
        self.minScore = 0
        for pos in self.board.maxPlayerTaken:
            self.maxScore += self.board.ptBoard[pos]
        for pos in self.board.minPlayerTaken:
            self.minScore += self.board.ptBoard[pos]
    
    def isComplete(self):
        '''
        Checks if the game is complete by check if there are open positions on the board.
        INPUTS: None
        OUTPUTS: None
        RETURN: returns true if and only if there are no open positions on the board and the game is complete.
        SIDE EFFECTS: None
        '''
        #if there are no open spaces
        if not self.board.openPositions:
            return True
        else:
            return False
        
    def runGame(self, strategy1 = 0, strategy2 = 0):
        '''
        Runs game
        INPUTS: strategy1 is the strategy of max, strategy2 is the strategy of min.
        OUTPUTS: None 
        RETURN: None
        SIDE EFFECTS: Changes the layout of the board object that was passed in. It also changes the score.
        '''
        #start with initial state
        player = 'max'
    
        #while the game is not complete
        while not self.isComplete():
            #max makes move with minimax
            if player == 'max' and strategy1 == 0:
                maxSeen = None
                move = None
                start = time.time()
                for openSpace in self.board.openPositions:
                    utility, nodes = self.minimax(openSpace, 'min', 1)
                    if maxSeen == None:
                        maxSeen = utility
                        move = openSpace
                    elif utility > maxSeen:
                        maxSeen = utility
                        move = openSpace
                    self.totNodesExpandedMax += nodes
                self.makeMove(move, player)
                end = time.time()
                self.totTimeMax += (end-start)
                self.totMovesMax += 1
                player = 'min'
        
            #min makes move with minimax
            elif player == 'min' and strategy2 == 0:
                minSeen = None
                move = None
                start = time.time()
                for openSpace in self.board.openPositions:
                    utility, nodes = self.minimax(openSpace, 'max', 1)
                    if minSeen == None:
                        minSeen = utility
                        move = openSpace
                    elif utility < minSeen:
                        minSeen = utility
                        move = openSpace
                    self.totNodesExpandedMin += nodes
                self.makeMove(move, player)
                end = time.time()
                self.totTimeMin += (end-start)
                self.totMovesMin += 1
                player = 'max'
            
            #max makes move with alpha-beta search
            elif player == 'max' and strategy1 == 1:
                alpha = None
                beta = None
                maxSeen = None
                move = None
                start = time.time()
                for openSpace in self.board.openPositions:
                    utility, nodes = self.alphaBetaSearch(openSpace, 'min', 1, alpha, beta)
                    if maxSeen == None:
                        maxSeen = utility
                        move = openSpace
                    elif utility > maxSeen:
                        maxSeen = utility
                        move = openSpace
                    self.totNodesExpandedMax += nodes
                self.makeMove(move, player)
                end = time.time()
                self.totTimeMax += (end-start)
                self.totMovesMax += 1
                player = 'min'
                
            #min makes move with alpha-beta search    
            else:
                alpha = None
                beta = None
                minSeen = None
                move = None
                start = time.time()
                for openSpace in self.board.openPositions:
                    utility, nodes = self.alphaBetaSearch(openSpace, 'max', 1, alpha, beta)
                    if minSeen == None:
                        minSeen = utility
                        move = openSpace
                    elif utility < minSeen:
                        minSeen = utility
                        move = openSpace
                    self.totNodesExpandedMin += nodes
                self.makeMove(move, player)
                end = time.time()
                self.totTimeMin += (end-start)
                self.totMovesMin += 1
                player = 'max'
                
        self.calculateScore()

    def minimax(self, pos, player, depth):
        '''
        Implements minimax search algorithm
        INPUTS: Potential next position to place piece, the current player, and the current depth in minimax
        OUTPUTS: None
        RETURNS: The minimum utility move if the current player in min, or the maximum utility move if the current player is 
        max. Also, returns the number of nodes expanded during a minimax search.
        '''
        num_nodes = 0
        maxDepth = 4
        conqueredPositions = self.makeMove(pos, player)
        if depth >= maxDepth or self.isComplete():
            self.reverseMove(pos, player, conqueredPositions)
            return self.utility(), 1
        
        #Do minimax for max player
        if player == 'max':
            maxSeen = None
            for openSpace in self.board.openPositions:
                utility, nodes = self.minimax(openSpace, 'min', depth + 1)
                if maxSeen == None:
                    maxSeen = utility
                    move = openSpace
                elif utility > maxSeen:
                    maxSeen = utility
                    move = openSpace
                num_nodes += nodes
            self.reverseMove(pos, player, conqueredPositions)
            return utility, num_nodes
        
        #Do minimax for min player
        else:
            minSeen = None
            for openSpace in self.board.openPositions:
                utility, nodes = self.minimax(openSpace, 'max', depth + 1)
                if minSeen == None:
                    minSeen = utility
                    move = openSpace
                elif utility < minSeen:
                    minSeen = utility
                    move = openSpace
                num_nodes += nodes
            self.reverseMove(pos, player, conqueredPositions)
            return utility, num_nodes
        
    def alphaBetaSearch(self, pos, player, depth, alpha = None, beta = None):
        '''
        Runs the alpha beta searching algorithms.
        INPUTS: potential positions of the player, the current player, the current depth of the algorithms, alpha
        (the best value seen by max so far), beta (the best value seen by min so far)
        OUTPUTS: none
        RETURNS: The minimum utility move if the current player in min, or the maximum utility move if the current player is max

        '''
        num_nodes = 0
        maxDepth = 5
        conqueredPositions = self.makeMove(pos, player)
        if depth >= maxDepth or self.isComplete():
            self.reverseMove(pos, player, conqueredPositions)
            return self.utility(), 1
        
        if player == 'max':
            maxSeen = None
            for openSpace in self.board.openPositions:
                utility, nodes = self.alphaBetaSearch(openSpace, 'min', depth + 1, alpha, beta)
                num_nodes += nodes
                if beta == None:
                    beta = utility
                elif utility >= beta:
                    self.reverseMove(pos, player, conqueredPositions)
                    return utility, num_nodes
                if maxSeen == None:
                    maxSeen = utility
                    alpha = utility
                    move = openSpace
                elif utility > maxSeen:
                    maxSeen = utility
                    alpha = utility
                    move = openSpace
            self.reverseMove(pos, player, conqueredPositions)
            return utility, num_nodes
        else:
            minSeen = None
            for openSpace in self.board.openPositions:
                utility, nodes = self.alphaBetaSearch(openSpace, 'max', depth + 1, alpha, beta)
                num_nodes += nodes
                if alpha == None:
                    alpha = utility
                elif utility <= alpha:
                    self.reverseMove(pos, player, conqueredPositions)
                    return utility, num_nodes
                if minSeen == None:
                    minSeen = utility
                    beta = utility
                    move = openSpace
                elif utility < minSeen:
                    minSeen = utility
                    beta = utility
                    move = openSpace
            self.reverseMove(pos, player, conqueredPositions)
            return utility, num_nodes
        
    def makeMove(self, move, player):
        '''
        Changes board by making move.
        INPUTS: position you want to make the move, player that will make the move.
        OUTPUTS: none
        RETURNS: returns the positions you conquered last.
        SIDE EFFECTS: Makes changes to the board corresponding to the move you made.
        '''
        positionsConquered = []
        
        #remove move from list of open positions
        if move in self.board.openPositions:
            idx = self.board.openPositions.index(move)
            del self.board.openPositions[idx]
        else:
            print "Invalid Move"
            return
        
        surroundingMax = self.checkForSurroundingMax(move)
        surroundingMin = self.checkForSurroundingMin(move)
        
        if player == 'max':
            self.board.maxPlayerTaken.append(move)
            
            #Blitz: Conquer min players piece
            if len(surroundingMax) != 0:
                for pos in surroundingMin:
                    idx = self.board.minPlayerTaken.index(pos)
                    del self.board.minPlayerTaken[idx]
                    self.board.maxPlayerTaken.append(pos)
                    positionsConquered.append(pos)
                
        elif player == 'min':
            self.board.minPlayerTaken.append(move)
            
            #Blitz: Conquer max players piece
            if len(surroundingMin) != 0:
                for pos in surroundingMax:
                    idx = self.board.maxPlayerTaken.index(pos)
                    del self.board.maxPlayerTaken[idx]
                    self.board.minPlayerTaken.append(pos)
                    positionsConquered.append(pos)
                    
        return positionsConquered
                    
    def reverseMove(self, pos, player, positionsConquered):
        '''
        Reverses a move made last.
        INPUTS: position of the last move you want to reverse, player that made the last move, positions conquered
        in the last move.
        OUTPUTS: none
        RETURNS: none
        SIDE EFFECTS: Reverses changes to the board
        '''
        if player == 'max':
            idx = self.board.maxPlayerTaken.index(pos)
            del self.board.maxPlayerTaken[idx]
            self.board.openPositions.append(pos)
            for tile in positionsConquered:
                idx = self.board.maxPlayerTaken.index(tile)
                del self.board.maxPlayerTaken[idx]
                self.board.minPlayerTaken.append(tile)
                
        else:
            idx = self.board.minPlayerTaken.index(pos)
            del self.board.minPlayerTaken[idx]
            self.board.openPositions.append(pos)
            for tile in positionsConquered:
                idx = self.board.minPlayerTaken.index(tile)
                del self.board.minPlayerTaken[idx]
                self.board.maxPlayerTaken.append(tile)
                        
    def checkForSurroundingMax(self, pos):
        '''
        Gets surrounding positions that the max player sits on.
        INPUTS: current position
        OUTPUTS: none
        RETURNS: surrounding positons that max sits on.
        SIDE EFFECTS: none
        '''
        surroundingMax = []
        for neighbor in self.getNeighbors(pos):
            if neighbor in self.board.maxPlayerTaken:
                surroundingMax.append(neighbor)
        return surroundingMax
        
    def checkForSurroundingMin(self, pos):
        '''
        Gets surrounding positions that min player sits on.
        INPUTS: current position
        OUTPUTS: none
        RETURNS: surrounding positons that min sits on.
        SIDE EFFECTS: none
        '''
        surroundingMin = []
        for neighbor in self.getNeighbors(pos):
            if neighbor in self.board.minPlayerTaken:
                surroundingMin.append(neighbor)
        return surroundingMin
            
    def getNeighbors(self, pos):
        '''
        Gets neighboring positons to your current position.
        INPUTS: current position
        OUTPUTS: none
        RETURNS: surrounding positions that max sits on.
        SIDE EFFECTS: none
        '''
        neighbors = []
        if pos % 6 != 0:
            neighbors.append(pos-1)
        if pos / 6 != 0:
            neighbors.append(pos-6)
        if pos % 6 != 5:
            neighbors.append(pos+1)
        if pos / 6 != 5:
            neighbors.append(pos+6)
        return neighbors

#Minimax Versus Minimax

In [208]:
newboard = board("game_boards/Keren.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [B] [B] [B] [B] [B]  
[B] [B] [B] [B] [B] [B]  
[A] [B] [A] [A] [A] [A]  
[B] [A] [A] [B] [A] [A]  
[B] [A] [B] [B] [B] [A]  
[A] [A] [A] [B] [B] [B]  
Max Score:  15
Min Score:  21
Average Time Per Move Max: 3.24586047067
Average Time Per Move Min: 2.95699369907
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 269824.388889


In [209]:
newboard = board("game_boards/Narvik.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[B] [B] [B] [A] [A] [A]  
[B] [B] [B] [A] [A] [B]  
[A] [A] [B] [A] [A] [A]  
[A] [B] [A] [B] [A] [B]  
[B] [B] [B] [A] [A] [A]  
[A] [B] [A] [A] [A] [A]  
Max Score:  707
Min Score:  1093
Average Time Per Move Max: 3.14023266898
Average Time Per Move Min: 2.80047429932
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 269824.388889


In [210]:
newboard = board("game_boards/Smolensk.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [B] [B] [A] [A] [A]  
[A] [B] [B] [B] [A] [A]  
[B] [B] [B] [B] [A] [A]  
[B] [B] [B] [B] [A] [A]  
[B] [B] [A] [B] [B] [A]  
[B] [A] [A] [A] [A] [A]  
Max Score:  669
Min Score:  984
Average Time Per Move Max: 3.06107362111
Average Time Per Move Min: 2.72686080138
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 269824.388889


In [211]:
newboard = board("game_boards/Westerplatte.txt")
newWarGame = warGame(newboard)
newWarGame.runGame()
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[B] [A] [A] [A] [A] [A]  
[A] [A] [B] [A] [B] [B]  
[B] [B] [B] [B] [B] [B]  
[B] [B] [B] [B] [A] [B]  
[A] [B] [B] [A] [A] [A]  
[A] [B] [B] [B] [A] [A]  
Max Score:  29
Min Score:  43
Average Time Per Move Max: 3.08663049009
Average Time Per Move Min: 2.80589638816
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 269824.388889


#Alpha-Beta Versus Minimax

In [212]:
newboard = board("game_boards/Keren.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,0)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [B] [B] [B] [B] [B]  
[B] [A] [B] [B] [B] [B]  
[A] [A] [A] [B] [B] [B]  
[A] [A] [A] [A] [A] [B]  
[A] [B] [A] [A] [A] [A]  
[B] [B] [B] [A] [A] [A]  
Max Score:  18
Min Score:  18
Average Time Per Move Max: 0.033934103118
Average Time Per Move Min: 2.77965705925
Average Nodes Expanded Per Move Max: 2159.22222222
Average Nodes Expanded Per Move Min: 269824.388889


In [213]:
newboard = board("game_boards/Narvik.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,0)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[B] [A] [B] [B] [A] [B]  
[A] [A] [A] [B] [B] [B]  
[B] [A] [A] [A] [B] [B]  
[B] [B] [A] [A] [A] [A]  
[B] [A] [A] [A] [B] [A]  
[A] [A] [A] [B] [A] [A]  
Max Score:  805
Min Score:  995
Average Time Per Move Max: 0.102795375718
Average Time Per Move Min: 2.80558727847
Average Nodes Expanded Per Move Max: 7216.83333333
Average Nodes Expanded Per Move Min: 269824.388889


In [214]:
newboard = board("game_boards/Smolensk.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,0)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [A] [A] [A] [A] [A]  
[A] [B] [A] [B] [A] [B]  
[B] [B] [B] [A] [A] [B]  
[B] [B] [B] [A] [A] [A]  
[B] [A] [B] [B] [A] [A]  
[B] [B] [B] [A] [A] [B]  
Max Score:  637
Min Score:  1016
Average Time Per Move Max: 0.349237442017
Average Time Per Move Min: 2.73215811782
Average Nodes Expanded Per Move Max: 28999.0
Average Nodes Expanded Per Move Min: 269824.388889


In [215]:
newboard = board("game_boards/Westerplatte.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,0)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [B] [A] [A] [B] [A]  
[B] [A] [A] [A] [A] [A]  
[A] [B] [A] [A] [A] [A]  
[B] [B] [A] [B] [A] [A]  
[B] [B] [B] [A] [A] [B]  
[B] [B] [A] [B] [B] [B]  
Max Score:  44
Min Score:  28
Average Time Per Move Max: 0.105349606938
Average Time Per Move Min: 2.72223614322
Average Nodes Expanded Per Move Max: 8407.55555556
Average Nodes Expanded Per Move Min: 269824.388889


#Minimax Versus Alpha-Beta

In [216]:
newboard = board("game_boards/Keren.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(0,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [A] [B] [B] [B] [A]  
[A] [B] [B] [B] [B] [B]  
[B] [A] [B] [B] [B] [A]  
[B] [A] [A] [B] [A] [B]  
[A] [A] [A] [A] [B] [A]  
[B] [A] [A] [A] [A] [A]  
Max Score:  19
Min Score:  17
Average Time Per Move Max: 3.04249542289
Average Time Per Move Min: 0.0201575623618
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 1238.05555556


In [217]:
newboard = board("game_boards/Narvik.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(0,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[B] [A] [A] [A] [B] [B]  
[A] [A] [A] [A] [A] [B]  
[B] [A] [A] [A] [B] [B]  
[B] [A] [A] [B] [B] [B]  
[A] [A] [A] [B] [B] [B]  
[A] [A] [A] [B] [B] [B]  
Max Score:  803
Min Score:  997
Average Time Per Move Max: 3.13592682944
Average Time Per Move Min: 0.0865450832579
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 6353.66666667


In [218]:
newboard = board("game_boards/Smolensk.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(0,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[B] [A] [B] [B] [B] [B]  
[A] [B] [B] [B] [A] [B]  
[A] [A] [B] [A] [B] [B]  
[A] [B] [A] [A] [A] [B]  
[B] [A] [A] [A] [B] [A]  
[A] [A] [A] [B] [B] [B]  
Max Score:  907
Min Score:  746
Average Time Per Move Max: 3.17459656133
Average Time Per Move Min: 0.341068519486
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 26850.8888889


In [219]:
newboard = board("game_boards/Westerplatte.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(0,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[B] [A] [B] [B] [A] [A]  
[A] [A] [B] [B] [B] [B]  
[A] [A] [B] [B] [A] [B]  
[A] [B] [B] [A] [A] [A]  
[A] [B] [B] [B] [A] [B]  
[A] [A] [B] [B] [B] [B]  
Max Score:  30
Min Score:  42
Average Time Per Move Max: 3.01859871546
Average Time Per Move Min: 0.0913686090046
Average Nodes Expanded Per Move Max: 311372.111111
Average Nodes Expanded Per Move Min: 7290.05555556


#Alpha-Beta Search Versus Alpha-Beta Search

In [220]:
newboard = board("game_boards/Keren.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [B] [A] [A] [A] [A]  
[B] [B] [B] [A] [B] [A]  
[A] [B] [A] [B] [A] [A]  
[A] [A] [A] [A] [B] [A]  
[A] [A] [B] [B] [B] [A]  
[A] [B] [B] [B] [B] [B]  
Max Score:  20
Min Score:  16
Average Time Per Move Max: 0.0176155964533
Average Time Per Move Min: 0.0183731847339
Average Nodes Expanded Per Move Max: 1172.33333333
Average Nodes Expanded Per Move Min: 1181.72222222


In [221]:
newboard = board("game_boards/Narvik.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [A] [B] [B] [B] [B]  
[A] [A] [A] [A] [B] [B]  
[A] [B] [B] [B] [B] [B]  
[B] [B] [B] [A] [B] [A]  
[A] [B] [B] [B] [B] [B]  
[A] [A] [A] [B] [B] [A]  
Max Score:  896
Min Score:  904
Average Time Per Move Max: 0.0830285549164
Average Time Per Move Min: 0.0856144295798
Average Nodes Expanded Per Move Max: 6411.44444444
Average Nodes Expanded Per Move Min: 6446.33333333


In [225]:
newboard = board("game_boards/Smolensk.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[B] [B] [B] [A] [A] [B]  
[A] [B] [B] [A] [A] [A]  
[B] [B] [A] [A] [A] [A]  
[B] [B] [B] [A] [A] [A]  
[B] [B] [A] [A] [A] [A]  
[A] [B] [B] [A] [B] [A]  
Max Score:  703
Min Score:  950
Average Time Per Move Max: 0.337630126211
Average Time Per Move Min: 0.310641010602
Average Nodes Expanded Per Move Max: 29628.3333333
Average Nodes Expanded Per Move Min: 26746.0


In [224]:
newboard = board("game_boards/Westerplatte.txt")
newWarGame = warGame(newboard)
newWarGame.runGame(1,1)
newWarGame.board.printBoard()
print "Max Score: ", newWarGame.maxScore 
print "Min Score: ", newWarGame.minScore
print "Average Time Per Move Max:", newWarGame.totTimeMax/newWarGame.totMovesMax
print "Average Time Per Move Min:", newWarGame.totTimeMin/newWarGame.totMovesMin
print "Average Nodes Expanded Per Move Max:", float(newWarGame.totNodesExpandedMax)/newWarGame.totMovesMax
print "Average Nodes Expanded Per Move Min:", float(newWarGame.totNodesExpandedMin)/newWarGame.totMovesMin

 
[A] [B] [A] [A] [B] [A]  
[A] [A] [A] [A] [B] [B]  
[A] [A] [A] [B] [B] [B]  
[A] [A] [A] [B] [B] [B]  
[A] [A] [B] [B] [A] [B]  
[A] [B] [B] [B] [B] [B]  
Max Score:  38
Min Score:  34
Average Time Per Move Max: 0.128967761993
Average Time Per Move Min: 0.117954942915
Average Nodes Expanded Per Move Max: 9093.16666667
Average Nodes Expanded Per Move Min: 8152.72222222
